# 图神经网络模型(一)

今天课堂主要讲解了三个部分：GCN 算法、GAT 算法、Message Passing 消息传递机制。

## GCN参数补充解释

主要是帮助大家理解消息传递机制的一些参数类型。

这里我们给出一个简化版本的 GCN 模型，帮助大家理解PGL框架实现消息传递的流程。

In [5]:
import paddle.fluid.layers as L

def gcn_layer(gw, feature, hidden_size, activation, name, norm=None):
    """
    描述：通过GCN层计算新的节点表示
    输入：gw - GraphWrapper对象
         feature - 节点表示 (num_nodes, feature_size)
         hidden_size - GCN层的隐藏层维度 int
         activation - 激活函数 str
         name - GCN层名称 str
         norm - 标准化tensor float32 (num_nodes,)，None表示不标准化
    输出：新的节点表示 (num_nodes, hidden_size)
    """

    # send函数
    def send_func(src_feat, dst_feat, edge_feat):
        """
        描述：用于send节点信息。函数名可自定义，参数列表固定
        输入：src_feat - 源节点的表示字典 {name:(num_edges, feature_size)}
             dst_feat - 目标节点表示字典 {name:(num_edges, feature_size)}
             edge_feat - 与边(src, dst)相关的特征字典 {name:(num_edges, feature_size)}
        输出：存储发送信息的张量或字典 (num_edges, feature_size) or {name:(num_edges, feature_size)}
        """
        return src_feat["h"] # 直接返回源节点表示作为信息

    # send和recv函数是搭配实现的，send的输出就是recv函数的输入
    # recv函数
    def recv_func(msg):
        """
        描述：对接收到的msg进行聚合。函数名可自定义，参数列表固定
        输出：新的节点表示张量 (num_nodes, feature_size)
        """
        return L.sequence_pool(msg, pool_type='sum') # 对接收到的消息求和

    ### 消息传递机制执行过程
    # gw.send函数
    msg = gw.send(send_func, nfeat_list=[("h", feature)]) 
    """ 
    描述：触发message函数，发送消息并将消息返回
    输入：message_func - 自定义的消息函数
         nfeat_list - list [name] or tuple (name, tensor)
         efeat_list - list [name] or tuple (name, tensor)
    输出：消息字典 {name:(num_edges, feature_size)}
    """

    # gw.recv函数
    output = gw.recv(msg, recv_func)
    """ 
    描述：触发reduce函数，接收并处理消息
    输入：msg - gw.send输出的消息字典
         reduce_function - "sum"或自定义的reduce函数
    输出：新的节点特征 (num_nodes, feature_size)

    如果reduce函数是对消息求和，可以直接用"sum"作为参数，使用内置函数加速训练，上述语句等价于 \
    output = gw.recv(msg, "sum")
    """

    # 通过以activation为激活函数的全连接输出层
    output = L.fc(output, size=hidden_size, bias_attr=False, act=activation, name=name)
    return output